## Import thư viện

In [ ]:
!pip install pyngrok

In [ ]:
!pip install streamlit

In [ ]:
import os
os.system('pip install /kaggle/input/kerasapplications -q')
os.system('pip install /kaggle/input/efficientnet-keras-source-code/ -q --no-deps')

In [ ]:
%%writefile app.py
## import
import streamlit as st
import tensorflow as tf
import streamlit as st
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt

## import model using
import efficientnet.tfkeras as efn
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19

## config
HEIGHT = 480
WIDTH = 480
CHANNELS = 3
CLASSES = 6
top_dropout_rate = 0.2
AUTO = tf.data.experimental.AUTOTUNE

weights_path_b0 = "../input/efficientnetb0/FGVC8-efn-b0.h5"
weights_path_vgg16 = "../input/fgvc8vgg16/FGVC8-VGG16.h5"
weights_path_vgg19 = "../input/fgvc8vgg19/FGVC8-VGG19t.h5"
weights_path_b7 = "../input/b7tpu/B7-tpu.h5"

## Get models
def get_model_b0():
    base_model = efn.EfficientNetB0(include_top=False, weights=None, input_shape=(HEIGHT, WIDTH, 3))

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(top_dropout_rate)(x)
    outputs = Dense(CLASSES, activation='sigmoid')(x)
    
    return Model(base_model.input, outputs)

def get_model_vgg19():
    VGG19_MODEL = tf.keras.applications.VGG19(weights=None ,include_top=False, input_shape=(HEIGHT, WIDTH, 3))
    
    x=VGG19_MODEL.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(256,activation='relu')(x)
    x=Dropout(top_dropout_rate)(x)
    x=Dense(128,activation='relu')(x)
    prediction=Dense(6,activation='sigmoid')(x)

    model=Model(inputs=VGG19_MODEL.input, outputs=prediction)
    
    return model

def get_model_vgg16():
    VGG16_MODEL = tf.keras.applications.VGG16(weights=None ,include_top=False, input_shape=(HEIGHT, WIDTH, 3))
    
    x=VGG16_MODEL.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(256,activation='relu')(x)
    x=Dropout(top_dropout_rate)(x)
    x=Dense(128,activation='relu')(x)
    prediction=Dense(6,activation='sigmoid')(x)

    model=Model(inputs=VGG16_MODEL.input, outputs=prediction)
    
    return model

## efn B7
def get_model_b7():
    model = tf.keras.models.Sequential(name='EfficientNetB7')
    
    model.add(efn.EfficientNetB7(
        include_top=False,
        input_shape=(768, 768, 3),
        weights=None,
        pooling='avg'))
    
    model.add(tf.keras.layers.Dense(CLASSES, 
        kernel_initializer=tf.keras.initializers.RandomUniform(seed=32),
        bias_initializer=tf.keras.initializers.Zeros(), name='dense_top'))
    model.add(tf.keras.layers.Activation('sigmoid', dtype='float32'))
    
    return model


## Load models
@st.cache(allow_output_mutation=True)
def load_model_b0():
    ## efn_b0
    model_b0 = get_model_b0()
    model_b0.load_weights(weights_path_b0)
    
    return model_b0

def load_model_vgg16():
    ## vgg16
    model_vgg16 = get_model_vgg16()
    model_vgg16.load_weights(weights_path_vgg16)
    
    return model_vgg16

def load_model_vgg19():
    ## vgg19
    model_vgg19 = get_model_vgg19()
    model_vgg19.load_weights(weights_path_vgg19)
    
    return model_vgg19

def load_model_b7():
    model_b7 = get_model_b7()
    model_b7.load_weights(weights_path_b7)
    
    return model_b7

with st.spinner('Model is being loaded..'):
    model_b0 = load_model_b0()
    model_vgg16 = load_model_vgg16()
    model_vgg19 = load_model_vgg19()
#     model_b7 = load_model_b7()
    
## Header
st.write("""
         # Plant Pathology 2021 - FGVC8
         """
         )

file = st.file_uploader("Please upload an brain scan file", type=["jpg", "png"])
import cv2
from PIL import Image, ImageOps
import numpy as np
st.set_option('deprecation.showfileUploaderEncoding', False)

## Augmentation
def data_full_augment(image):
    
    p_spatial = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_rotate = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_1 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_2 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    p_pixel_3 = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    
    flag = False

    # Pixel-level transforms
    if p_pixel_1 >= .4:
        image = tf.image.random_saturation(image, lower=.7, upper=1.3)
        flag = True
    if p_pixel_2 >= .4:
        image = tf.image.random_contrast(image, lower=.8, upper=1.2)
        flag = True
    if p_pixel_3 >= .4:
        image = tf.image.random_brightness(image, max_delta=.1)
        flag = True


    # Flips
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    if p_spatial > .75:
        image = tf.image.transpose(image)
        flag = True
        
    # Rotates
    if p_rotate > .75:
        image = tf.image.rot90(image, k=3) # rotate 270º
        flag = True
    elif p_rotate > .5:
        image = tf.image.rot90(image, k=2) # rotate 180º
        flag = True
    elif p_rotate > .25:
        image = tf.image.rot90(image, k=1) # rotate 90
        flag = True
        
    # Crops
    if p_crop > .7:
        if p_crop > .9:
            image = tf.image.central_crop(image, central_fraction=.7)
        elif p_crop > .8:
            image = tf.image.central_crop(image, central_fraction=.8)
        else:
            image = tf.image.central_crop(image, central_fraction=.9)
        flag = True
    elif p_crop > .4:
        HEIGHT1 = image.shape[0]
        WIDTH1 = image.shape[1]
        crop_size_h = tf.random.uniform([], int(HEIGHT1*.8), HEIGHT1, dtype=tf.float32)
        crop_size_w = tf.random.uniform([], int(WIDTH1*.8), WIDTH1, dtype=tf.float32)
        image = tf.image.random_crop(image, size=[crop_size_h, crop_size_w, 3])
        flag = True
    
    return image

def process(img):
    img = data_full_augment(img)
    img = tf.cast(img, tf.float32) / 255.0
    img = np.array(img)
    return cv2.resize(img , (480, 480)).reshape(-1, 480, 480, 3)

def extract(preds, thres):
    preds = np.array(preds)
    preds = (preds>thres)
    n_label = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
    temp = []
    for i, k in enumerate(n_label):
        if preds[i]:
            temp.append(k)
    if len(temp) > 1 and "healthy" in temp:
        temp.remove("healthy")
    elif len(temp) == 0:
        temp.append('healthy')
    
    return " ".join(temp)

## import and predict
def import_and_predict(image_data, model_b0, model_vgg16, model_vgg19):
        image = np.asarray(image_data)
        image = process(image)
        
        result = {}
        result['models'] = ['EfficientNet B0', 'VGG16', 'VGG19']
        
        ##predict efn b0
        pred_b0 = model_b0.predict(image)[0]
        output_b0 = extract(pred_b0, 0.3)
        
        ##predict vgg16
        pred_vgg16 = model_vgg16.predict(image)[0]
        output_vgg16 = extract(pred_vgg16, 0.27)
        
        ##predict vgg19
        pred_vgg19 = model_vgg19.predict(image)[0]
        output_vgg19 = extract(pred_vgg19, 0.27)
        
        ##predict efn b7
#         pred_b7 = model_b7.predict(image)[0]
#         output_b7 = extract(pred_b7, 0.35)
        
        result['result'] = [output_b0, output_vgg16, output_vgg19]
        
        df_result = pd.DataFrame(data=result)
        
        return df_result
    

if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    df_result = import_and_predict(image, model_b0, model_vgg16, model_vgg19, model_b7)
    
    st.write("""
         ## Kết quả:
         """
         )
    
    st.dataframe(data=df_result)

In [ ]:
!ngrok authtoken 21igYrFjznjBBTZbCPStCsjKsMi_NBeR4LsfyZakDoNZKzfg

In [ ]:
from pyngrok import ngrok
 
public_url = ngrok.connect('8501')
public_url

In [ ]:
!streamlit run app.py >/dev/null